In [ ]:
def state_batch_right_dim1(state_batch):
    states_batch, state_action_values, i = [], [], 0
    while i<len(state_batch):
        state = state_batch[i:i+4]
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
        states_batch.append(state)
        state_action_value=policy_net(state)
        state_action_values.append(state_action_value)
        i+=4
    return torch.cat(state_action_values)

In [ ]:
def state_batch_right_dim11(non_final_next_states):
    non_final_next_statess, i = [], 0
    while i<len(non_final_next_states):
        state_None = non_final_next_states[i:i+4]
        state_None = torch.tensor(state_None, dtype=torch.float32, device=device).unsqueeze(0)
        non_final_next_state = target_net(state_None).max(1)[0].detach()
        non_final_next_statess.append(non_final_next_state)
        i+=4
    return torch.cat(non_final_next_statess)

In [ ]:
def optimize_model():
    #unzip batch transitions // calculate q values/ next q values/ and target q values
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    #account for the next zero q values (if life is terminated)

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                        batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                       if s is not None])
    state_action_values = state_batch_right_dim1(state_batch).gather(1, action_batch)  #q_values #(q_values(shape==32), 1)
    print("state_action_values", state_action_values)
    epsilon = 0.05
    next_state_values = torch.zeros(32, device=device) #32==batch_size

    with torch.no_grad():
        if non_final_mask.any():
            next_state_values[non_final_mask] = state_batch_right_dim11(non_final_next_states)

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    #torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()